# Set up
Uses MySQL.
See README.txt to set up database.

Run Import MySQL section first before running anything else.

# Guide
Above each section of code is a markdown that explains what that section does.
If there are multiple cells, then above each cell there will be information on what that cell does.
Often one will be for inputs and the other will be for executing the query.
The input cell is required to be run first.

When a code block is ran, run it to completion.

## Import MySQL
### Run these to connect to mysql

Run this cell if mysql-connector-python not already installed.
No problems occur if it is already installed and this is run.

In [ ]:
# run if connector not already installed
# no error if already satisfied
!pip install mysql-connector-python

Make the connection with required username and password inputs.

In [ ]:
# connects to mysql
import mysql.connector

# get DB username and password from the standard input
import getpass
db_user = getpass.getpass('Enter username')
db_password = getpass.getpass('Enter password')

# prepare the configuration parameters for connecting to database
config = {
   'user': db_user,
   'password': db_password,
   'host': 'localhost',
   'port': 3306,
   'database': 'PlantNursery',
   'raise_on_warnings': True                    
}

#connect to DB server
db_connection = mysql.connector.connect(**config)

## Load sample data
### Load sample data for testing

In [ ]:
cur = db_connection.cursor()
# Open and read the file
fd = open('aaron_reamer_loadAll.sql', 'r')
sqlFile = fd.read()
fd.close()
# split into individual commands
sqlLoads = sqlFile.split(';')
del sqlLoads[-1] #delete last empty item
# run each
for load in sqlLoads:
    cur.execute(load)
db_connection.commit()
cur.close()

## Retrieve
### Retrieve requested data.

#### Plants

In [ ]:
cur = db_connection.cursor()
query = ('select * from plants')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()  

#### Pots

In [ ]:
cur = db_connection.cursor()
query = ('select * from pots order by pot_id asc')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()

#### Pot Log Entries
Orders by pot and then most recent first.

In [ ]:
cur = db_connection.cursor()
query = ('select * from log_entry order by l_id desc')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()

#### Pots with plants

In [ ]:
cur = db_connection.cursor()
query = ('select * from (plants natural inner join holds) natural inner join pots order by pot_id asc')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()

#### Trays

In [ ]:
cur = db_connection.cursor()
query = ('select * from tray order by tray_id asc')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()

#### Trays with pots

In [ ]:
cur = db_connection.cursor()
query = ('select * from (pots natural inner join containing) natural inner join tray order by tray_id asc')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()

#### Weather stations

In [ ]:
cur = db_connection.cursor()
query = ('select * from micro_weather_station tray order by station_id asc')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()

#### Measurements

In [ ]:
cur = db_connection.cursor()
query = ('select * from measurements order by station_id asc')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")

db_connection.commit()
cur.close()

## Insert and Update
Each section below contains an input section, an insert section, and an update section.
Does not include log entries or measurements which are covered separately.

input: inputs for all data

insert: will insert the input data

update: will update one variable at a time based on separate inputs

### Plants

#### input

In [ ]:
c_name = '"'+input("common name: ")+'"'
spec = '"'+input("species: ")+'"'
vri = '"'+input("variety: ")+'"'
pt = '"'+input("plant type: ")+'"'
ret = '"'+input("retention: ")+'"'
ger = input("germination days: ")
fd = input("required food: ")
wt = input("required water: ")
temp = input("temperature: ")
mois = input("moisture: ")
lig = input("light: ")
desc = input("description: ")

#### insert

In [ ]:
cur = db_connection.cursor()
query = 'insert into plants values({},{},{},{},{},{},{},{},{},{},{},{})'.format(c_name, spec, vri, pt, ret, ger, fd, wt, temp, mois, lig, desc)
cur.execute(query)
db_connection.commit()
cur.close()

#### update
Also used for relocating

In [ ]:
print("Input the common name of what you would like to update:")
p_id = '"'+input("common name: ")+'"'
print("Input variable you would like to update.\noptions:")
print("common_name")
print("species")
print("variety")
print("p_type")
print("retention")
print("germination_days")
print("feed")
print("water")
print("temperature")
print("moisture")
print("light")
print("descript")
set_up = input("variable: ")
print("Input the change:")
change = input("change: ")
#account for string
if set_up == "common_name" or set_up == "species" or set_up == "variety" or set_up == "p_type" or set_up == "retention":
    change = '"'+change+'"'
    
#update
cur = db_connection.cursor()
query = 'update plants set {} = {} where common_name = {}'.format(set_up,change,p_id)
cur.execute(query)
db_connection.commit()
cur.close()

### Pots

#### input

In [ ]:
potId = '"'+input("pot ID (8 characters): ")+'"'
vol = input("volume: ")
height = input("height: ")

#### insert

In [ ]:
cur = db_connection.cursor()
query = 'insert into pots values({},{},{})'.format(potId, vol, height)
cur.execute(query)
db_connection.commit()
cur.close()

#### update

In [ ]:
print("Input the pot ID of what you would like to update:")
pt_id = '"'+input("pot ID: ")+'"'
print("Input variable you would like to update.\noptions:")
print("pot_id")
print("volume")
print("height")
pset_up = input("variable: ")
print("Input the change:")
pchange = input("change: ")
if pset_up == "pot_id":
    pchange = '"'+pchange+'"'
    
#update
cur = db_connection.cursor()
query = 'update pots set {} = {} where pot_id = {}'.format(pset_up,pchange,pt_id)
cur.execute(query)
db_connection.commit()
cur.close()

### Trays

#### input

In [ ]:
trayId = '"'+input("tray ID (8 characters): ")+'"'
locx = input("location x: ")
locy = input("location y: ")

#### insert

In [ ]:
cur = db_connection.cursor()
query = 'insert into tray values({},point({},{}), current_time)'.format(trayId, locx, locy)
cur.execute(query)
db_connection.commit()
cur.close()

#### update
Also used for relocating

In [ ]:
print("Input the tray ID of what you would like to update:")
pt_id = '"'+input("tray ID: ")+'"'
print("Input variable you would like to update.\noptions:")
print("tray_id")
print("location\t(format: x,y)") # relocating tray
print("timestamp\t(format: YYYY-MM-DD hh:mm:ss)")
pset_up = input("variable: ")
print("Input the change:")
pchange = input("change: ")
if pset_up == "tray_id" or pset_up == "timestamp":
    pchange = '"'+pchange+'"'
elif pset_up == "location":
    pchange = "point("+pchange+")"
    
#update
cur = db_connection.cursor()
query = 'update tray set {} = {} where tray_id = {}'.format(pset_up,pchange,pt_id)
cur.execute(query)
db_connection.commit()
cur.close()

### Weather Stations

#### input

In [ ]:
staId = '"'+input("station ID (8 characters): ")+'"'
locx = input("location x: ")
locy = input("location y: ")

#### insert

In [ ]:
cur = db_connection.cursor()
query = 'insert into micro_weather_station values({},point({},{}))'.format(staId, locx, locy)
cur.execute(query)
db_connection.commit()
cur.close()

#### update

In [ ]:
print("Input the station ID of what you would like to update:")
pt_id = '"'+input("station ID: ")+'"'
print("Input variable you would like to update.\noptions:")
print("station_id")
print("location\t(format: x,y)") #relocating station
pset_up = input("variable: ")
print("Input the change:")
pchange = input("change: ")
if pset_up == "tray_id" or pset_up == "timestamp":
    pchange = '"'+pchange+'"'
elif pset_up == "location":
    pchange = "point("+pchange+")"
    
#update
cur = db_connection.cursor()
query = 'update micro_weather_station set {} = {} where station_id = {}'.format(pset_up,pchange,pt_id)
cur.execute(query)
db_connection.commit()
cur.close()

## Delete
Delete entries based off inputs

### Plants

In [ ]:
print("Input identifier to delete:")
del_id = '"'+input("common name: ")+'"'
cur = db_connection.cursor()
query = 'delete from plants where common_name = {}'.format(del_id)
cur.execute(query)
db_connection.commit()
cur.close()

### Pots and their logs

In [ ]:
print("Input identifier to delete:")
del_id = '"'+input("pot ID: ")+'"'
cur = db_connection.cursor()
query = 'delete from pots where pot_id = {}'.format(del_id)
cur.execute(query)
query = 'delete from log_entry where pot_id = {}'.format(del_id)
cur.execute(query)
db_connection.commit()
cur.close()

### Trays

In [ ]:
print("Input identifier to delete:")
del_id = '"'+input("tray ID: ")+'"'
cur = db_connection.cursor()
query = 'delete from tray where tray_id = {}'.format(del_id)
cur.execute(query)
db_connection.commit()
cur.close()

### Weather stations and their measurements

In [ ]:
print("Input identifier to delete:")
del_id = '"'+input("station ID: ")+'"'
cur = db_connection.cursor()
query = 'delete from micro_weather_station where station_id = {}'.format(del_id)
cur.execute(query)
db_connection.commit()
cur.close()

## Plant in Pot
Plant a plant into a pot.

In [ ]:
ptId = '"'+input("Pot ID: ")+'"'
plCN = '"'+input("Plant Common Name: ")+'"'
print("Date. Input 'CD' current Date.\nFormat YYYY-MM-DD")
pDate = '"'+input("Date: ")+'"'

if pDate == '"CD"':
    pDate = "curdate()"

cur = db_connection.cursor()
query = 'call insert_holds({}, {}, {});'.format(ptId, plCN, pDate)
cur.execute(query)
db_connection.commit()
cur.close()

## Empty Pot
Remove plant from a pot.

In [ ]:
ptId = '"'+input("Pot ID: ")+'"'

cur = db_connection.cursor()
query = 'delete from pots where pot_id = {}'.format(ptId)
cur.execute(query)
db_connection.commit()
cur.close()

## Move a pot to a tray
Moves a pot to a tray. Will not move it if tray is full. Also used for inserting pots that don't have a tray.

In [ ]:
ptId = '"'+input("Pot ID: ")+'"'
trId = '"'+input("Tray ID: ")+'"'

cur = db_connection.cursor()
query = 'call move_pot({}, {});'.format(trId,ptId)
cur.execute(query)
db_connection.commit()
cur.close()

## Remove a pot from a tray

In [ ]:
potId = '"'+input("Pot ID: ")+'"'

cur = db_connection.cursor()
query = 'call remove_pot({});'.format(potId)
cur.execute(query)
db_connection.commit()
cur.close()

## Take Measurement

In [ ]:
sId = '"'+input("Station ID: ")+'"'
print("Format: YYYY-MM-DD hh:mm:ss\nLeave blank for current time.")
tm = '"'+input("Timestamp: ")+'"'
tem = input("Temperature: ")
mos = input("Moisture: ")
lt = input("Ambient Light: ")

if tm == '""':
    tm = "CURRENT_TIMESTAMP"

cur = db_connection.cursor()
query = 'call take_measurement({}, {}, {}, {}, {});'.format(sId, tm, tem, mos, lt)
cur.execute(query)
db_connection.commit()
cur.close()

## Care for individual pot
Cares for a pot and adds a log entry

In [ ]:
potId = '"'+input("Pot ID: ")+'"'
print("Format: YYYY-MM-DD hh:mm:ss\nLeave blank for current time.")
tm = '"'+input("Timestamp: ")+'"'
fd = input("Food: ")
wt = input("Water: ")

if tm == '""':
    tm = "CURRENT_TIMESTAMP"

cur = db_connection.cursor()
query = 'call record_log({}, {}, {}, {});'.format(potId, tm, fd, wt)
cur.execute(query)
db_connection.commit()
cur.close()

## Care for entire tray
Cares for all pots in a tray and adds appropriate log entry.
Feeds and waters all pots in a tray.

In [ ]:
trayId = '"'+input("Tray ID: ")+'"'
print("Format: YYYY-MM-DD hh:mm:ss\nLeave blank for current time.")
tm = '"'+input("Timestamp: ")+'"'
fd = input("Food: ")
wt = input("Water: ")

if tm == '""':
    tm = "CURRENT_TIMESTAMP"

cur = db_connection.cursor()
query = 'call tray_care({}, {}, {}, {});'.format(trayId, tm, fd, wt)
cur.execute(query)
db_connection.commit()
cur.close()

## Special queries
special queries for this assignment

List the unique common names of all thevegetables currently planted.

In [ ]:
cur = db_connection.cursor()
query = ('call vegetables_planted();')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close()  

List the species name and number of varieties for each plant species for all the plants.

In [ ]:
cur = db_connection.cursor()
query = ('call num_varieties();')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

Find the total number of pots currently holding a plant with a given common name.

In [ ]:
cn = '"'+input("Common Name: ")+'"'

cur = db_connection.cursor()
query = ('call pots_with({});').format(cn)
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

Find the histogram (value and count) of volumes of pots with germinated plants with a given common name.

In [ ]:
cn = '"'+input("Common Name: ")+'"'

cur = db_connection.cursor()
query = ('call volume_histogram({});').format(cn)
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

Find the most populous species among the pots with germinated plants.

In [ ]:
cur = db_connection.cursor()
query = ('call most_germinated();')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

Find the oldest (in terms of current age) vegetable plant(s) among those that germinated during the previous month.

In [ ]:
cur = db_connection.cursor()
query = ('call last_month_oldest();')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

Find the germinated plant(s) that received the most daily water (averaged over their age).

In [ ]:
cur = db_connection.cursor()
query = ('call most_daily_water();')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

Find the number of planted herbs that received more food than the average amount of food received by the germinated vegetables.

In [ ]:
cur = db_connection.cursor()
query = ('call food_much();')
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

Find the germinated flowers that received more dailyambient light in a given month than the previous month.

In [ ]:
month = input("Month: ")

cur = db_connection.cursor()
query = ('call month_more_light({});').format(month)
cur.execute(query)
out = cur.fetchall()
for row in out:
    print(row)
    print ("\n")
    
cur.close() 

## Drop All
### Drops entire Database

In [ ]:
cur = db_connection.cursor()
# Open and read the file
fd = open('aaron_reamer_dropAll.sql', 'r')
sqlFile = fd.read()
fd.close()
# split into individual commands
sqlDrops = sqlFile.split(';')
# run each
for drop in sqlDrops:
    cur.execute(drop)
    
cur.close()

## Close Connection
### Do this before exiting.

In [ ]:
db_connection.close()